In [78]:
import os.path
from os import path
import pandas as pd

# 캐글 ↓제출경로
data_path = '/kaggle/input/kdt-ai-1/'
if not path.exists(data_path):
    data_path = './input/COVID_19_XRAY/'

train_path = data_path + 'train/'
test_path = data_path + 'test/'

from glob import glob

print(f'훈련 데이터 경로 : {train_path}')
print(f'테스트 데이터 경로 : {test_path}')
print(f'훈련 데이터 개수 : {len(glob(train_path + "*.png"))}')
print(f'테스트 데이터 개수 : {len(glob(test_path + "*.png"))}')

훈련 데이터 경로 : ./input/COVID_19_XRAY/train/
테스트 데이터 경로 : ./input/COVID_19_XRAY/test/
훈련 데이터 개수 : 0
테스트 데이터 개수 : 0


In [79]:
# 시드값 및 고정 GPU 장비 설정 

import torch
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [80]:
# 데이터 증강을 위한 이미지 변환기 정의
from torchvision import transforms 

# 훈련 데이터용 변환기
tranform_train = transforms.Compose([
    transforms.Resize((250,250)), # 이미지 크기 조정
    transforms.CenterCrop(180), # 중앙 이미지 확대
    transforms.RandomHorizontalFlip(0.5), # 좌우 대칭
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.465, 0.406),
                         (0.229, 0.224, 0.225)) # 정규화
    
])

tranform_test = transforms.Compose([
    transforms.Resize((250,250)), # 이미지 크기 조정
    transforms.CenterCrop(180), # 중앙 이미지 확대
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.465, 0.406),
                         (0.229, 0.224, 0.225)) # 정규화
    
])

In [81]:
from torchvision.datasets import ImageFolder

# 훈련 데이터
# Image Folder 를 사용하기 위해서 타깃값을 같은 디렉토리 안에서 사용했다.   
datasets_train = ImageFolder(root=train_path, transform=tranform_train)

In [82]:
# 데이터 로더의 시드값 고정 
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() %2 **32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
# 제네레이더 시드값 고정
g = torch.Generator()
g.manual_seed(0)

In [83]:
# 데이터 로더 만들기

from torch.utils.data import DataLoader

batch_size = 8

loader_train = DataLoader(dataset=datasets_train, batch_size=batch_size,
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g, num_workers=0)
                                    # 성능 안좋아서 num_workes=0으로 함 원래는 2임 
# 배치크기는 굳이 8이 아니어도 됨

In [84]:
# EfficicentNet 을 사용하겠음
!pip install efficientnet-pytorch==0.7.1

In [85]:
from efficientnet_pytorch import EfficientNet
# https://arxiv.org/abs/1905.11946 : 논문링크
# 모델 생성
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
# 베이스라인의 가장 간단한 efficientnet-b0 이진분류이므로 num_classes=2

# 장비 할당
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


In [86]:
print('모델 파라미터 개수 : ', sum(param.numel() for param in model.parameters()))

모델 파라미터 개수 :  4010110


In [87]:
# 모델 훈련 및 성능 검증 

import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [88]:
# 훈련함수 작성 
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm

def train(model, loader_train, loader_valid ,criterion, optimizer, scheduler=None, epochs=10, 
          save_file='model_state_dict.pth'):
    valid_loss_min = np.inf
    # 총 epoch 만큼 반복
    for epoch in range(epochs):
        print(f'에폭 [{epoch+1}/{epochs}] \n---------------------')
        # == [훈련] =========
        model.train() # 모델을 훈련 상태로 설정
        epoch_train_loss = 0 # 에폭별 손실값 초기화(훈련데이터 용)
        
        # 미니배치 단위로 훈련
        for images , labels in tqdm(loader_train):
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            epoch_train_loss += loss.item()
            
            loss.backward()
            optimizer.step() 
            
            if scheduler != None: # 스케줄러 학습률 갱신
                scheduler.step()
                
        print(f'\t 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
        
        # == [ 검증 ] ======================================================
        
        model.eval()
        epoch_valid_loss = 0
        preds_list = []
        true_list = []
        
        with torch.no_grad():
            # 하나의 에폭만큼의 데이터를 미니배치 단위로 검증
            
            for images, labels in loader_valid:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_valid_loss += loss.item()
                
                # 예측값 및 실젯값
                preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
                true = labels.cpu().numpy()
                
                preds_list.extend(preds)
                true_list.extend(true)

        # 현재 에폭의 검증 완료
        
        print(f'\t 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
        
        # 평가 지표 계산(정확도, 재현율 , F1 점수)
        val_accuracy = accuracy_score(true_list, preds_list)
        val_recall = recall_score(true_list, preds_list)
        val_f1_score = f1_score(true_list, preds_list)
        print(f'\t 정확도 : {val_accuracy: .4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')
        
        
        # 
            # 기울기 초기화
            # 순전파
            # 손실값 계산(훈련 데이터용)
            # 역전파
            # 가중치 갱신
            # 학습률 갱신
        
        if epoch_valid_loss <= valid_loss_min:
            print(f'\t### 검증 데이터 손실값 감소 ({valid_loss_min:.4f} --> {epoch_valid_loss:.4f}). 모델저장')
            torch.save(model.state_dict(), save_file)
            valid_loss_min = epoch_valid_loss
        # == [최적 모델 가중치 찾기] ==
        # 현 에폭에서의 검증 데이터 손실값이 지금까지 가장 작다면
            # 현 에폭의 모델 가중치 (현재까지의 최적 모델 가중치 ) 저장
    return torch.load(save_file)

# 조기종료
# 평가점수가 더 이상 좋아지지 않으면 훈련을 멈춪도록 

In [89]:
datasets_valid = ImageFolder(root="./input/COVID_19_XRAY/valid/", transform=tranform_test)

loader_valid = DataLoader(dataset=datasets_valid, batch_size=batch_size,
                          shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=0)


In [90]:
model_state_dict = train(
    model=model,
    loader_train=loader_train,
    loader_valid=loader_valid,
    criterion=criterion,
    optimizer=optimizer
)

에폭 [1/10] 
---------------------


  0%|          | 0/225 [00:00<?, ?it/s]

	 훈련 데이터 손실값 : 0.7828
	 검증 데이터 손실값 : 34.7250
	 정확도 :  0.4950 / 재현율 : 0.0300 / F1 점수 : 0.0561
	### 검증 데이터 손실값 감소 (inf --> 868.1259). 모델저장
에폭 [2/10] 
---------------------


  0%|          | 0/225 [00:00<?, ?it/s]

	 훈련 데이터 손실값 : 0.6519
	 검증 데이터 손실값 : 0.8520
	 정확도 :  0.7550 / 재현율 : 0.8300 / F1 점수 : 0.7721
	### 검증 데이터 손실값 감소 (868.1259 --> 21.2996). 모델저장
에폭 [3/10] 
---------------------


...

	 훈련 데이터 손실값 : 0.4470
	 검증 데이터 손실값 : 1.2831
	 정확도 :  0.7300 / 재현율 : 0.9100 / F1 점수 : 0.7712
에폭 [9/10] 
---------------------


  0%|          | 0/225 [00:00<?, ?it/s]

	 훈련 데이터 손실값 : 0.4396
	 검증 데이터 손실값 : 2.4433
	 정확도 :  0.5050 / 재현율 : 0.0200 / F1 점수 : 0.0388
에폭 [10/10] 
---------------------


  0%|          | 0/225 [00:00<?, ?it/s]

	 훈련 데이터 손실값 : 0.4380
	 검증 데이터 손실값 : 0.6137
	 정확도 :  0.8350 / 재현율 : 0.9500 / F1 점수 : 0.8520


In [91]:
datasets_test = ImageFolder(root=test_path, transform =tranform_test) 

loader_test = DataLoader(dataset=datasets_test, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=0)

In [92]:
def predict(model, loader_test, return_true=False):
    model.eval()
    preds_list = []
    true_list = []
    
    with torch.no_grad():
        for images, labels in loader_test:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            
            preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
            true = labels.cpu().numpy()
            
            preds_list.extend(preds)
            true_list.extend(true)
            
    if return_true:
        return true_list, preds_list
    else:
        return preds_list

In [93]:
true_list, preds_list = predict(model=model,
                                loader_test=loader_test,
                                return_true=True)

In [94]:
print(true_list)
print(preds_list)

original_covid_list = preds_list
print("before")
print(preds_list)
original_covid_list = [1] + original_covid_list
original_covid_list = original_covid_list[:-1]
print(len(original_covid_list)) 
print("after")
print(original_covid_list)
# 1=> normal
# 0=> covid 로 예측함

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [51]:
answer = []
for tmp in original_covid_list:
    if tmp ==0 :
        answer.append('covid')
    else:
        answer.append('normal')
print(answer)
print(len(answer))

['normal', 'covid', 'normal', 'covid', 'normal', 'normal', 'covid', 'covid', 'normal', 'covid', 'normal', 'normal', 'normal', 'normal', 'normal', 'covid', 'normal', 'normal', 'covid', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'covid', 'normal', 'normal', 'normal', 'normal', 'covid', 'normal', 'normal', 'normal', 'normal', 'covid', 'normal', 'covid', 'covid', 'normal', 'normal', 'normal', 'normal', 'normal', 'covid', 'normal', 'covid', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'covid', 'covid', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'covid', 'normal', 'normal', 'normal', 'covid', 'normal', 'normal', 'normal', 'covid', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'covid', 'normal', 'normal', 'normal', 'covid', 'normal', 'normal', 'covid', 'normal', 'normal', 'normal', 'normal', 'covid', 'covid', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'no

In [95]:
submission_df = pd.read_csv('submission.csv')

In [96]:
submission_df['label']= answer

In [97]:
submission_df.head()

,filename,label
0,image_001.png,normal
1,image_002.png,covid
2,image_003.png,normal
3,image_004.png,covid
4,image_005.png,normal


In [57]:
submission_df.tail()

,filename,label
395,image_396.png,normal
396,image_397.png,normal
397,image_398.png,normal
398,image_399.png,normal
399,image_400.png,normal


In [98]:
submission_df.to_csv("submission.csv",index=False)

submission_df.head()


,filename,label
0,image_001.png,normal
1,image_002.png,covid
2,image_003.png,normal
3,image_004.png,covid
4,image_005.png,normal


In [99]:
# https://www.kaggle.com/docs/api 참고
# kaggle competitions submit -c [COMPETITION] -f [FILE] -m [MESSAGE]

import os
import subprocess

# 0. api 토큰 발급받은이후 올바른 경로에 생성된 kaggle.json 추가
# 참고 url : https://github.com/Kaggle/kaggle-api
# cmd 명령어 실행으로 다운받기
result = subprocess.check_output(
    "kaggle competitions submit -c kdtai-1 -f ./submission.csv -m submit_in_local", 
    encoding='utf-8'
    )
print(result)

Successfully submitted to KDT_AI-프로젝트공모전스터디-1과제


In [64]:
model_list = []

from efficientnet_pytorch import EfficientNet

# 모델 생성
efficientnet_b1 = EfficientNet.from_pretrained('efficientnet-b1', num_classes=2)
efficientnet_b2 = EfficientNet.from_pretrained('efficientnet-b2', num_classes=2)
efficientnet_b3 = EfficientNet.from_pretrained('efficientnet-b3', num_classes=2)

# 장비할당

efficientnet_b1 = efficientnet_b1.to(device)
efficientnet_b2 = efficientnet_b2.to(device)
efficientnet_b3 = efficientnet_b3.to(device)

# 리스트에 모델 저장

model_list.append(efficientnet_b1)
model_list.append(efficientnet_b2)
model_list.append(efficientnet_b3)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b1-f1951068.pth" to C:\Users\user/.cache\torch\hub\checkpoints\efficientnet-b1-f1951068.pth


  0%|          | 0.00/30.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b1


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to C:\Users\user/.cache\torch\hub\checkpoints\efficientnet-b2-8bb594d6.pth


  0%|          | 0.00/35.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b2


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to C:\Users\user/.cache\torch\hub\checkpoints\efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


In [66]:
for idx, model in enumerate(model_list):
    num_params = sum(param.numel() for param in model.parameters())
    print(f'모델 {idx+1} 파라미터 개수 : {num_params}')

모델 1 파라미터 개수 : 6515746
모델 2 파라미터 개수 : 7703812
모델 3 파라미터 개수 : 10699306


In [67]:
# 손실함수 , 옵티마이저 ,스케줄러 설정 

import torch.nn as nnn

criterion = nn.CrossEntropyLoss()

optimizer1 = torch.optim.AdamW(model_list[0].parameters(), lr=0.0006, weight_decay=0.001)
optimizer2 = torch.optim.AdamW(model_list[1].parameters(), lr=0.0006, weight_decay=0.001)
optimizer3 = torch.optim.AdamW(model_list[2].parameters(), lr=0.0006, weight_decay=0.001)

In [68]:
from transformers import get_cosine_schedule_with_warmup

epochs = 20

# 스케줄러
scheduler1 = get_cosine_schedule_with_warmup(optimizer1,
                                             num_warmup_steps=len(loader_train)*3,
                                             num_training_steps=len(loader_train)*epochs)

scheduler2 = get_cosine_schedule_with_warmup(optimizer2,
                                             num_warmup_steps=len(loader_train)*3,
                                             num_training_steps=len(loader_train)*epochs)

scheduler3 = get_cosine_schedule_with_warmup(optimizer3,
                                             num_warmup_steps=len(loader_train)*3,
                                             num_training_steps=len(loader_train)*epochs)


In [77]:
# 첫번 쨰 모델 훈련 

model_state_dict = train(model=model_list[0],
                         loader_train=loader_train,
                         loader_valid=loader_valid,
                         criterion=criterion,
                         optimizer=optimizer1,
                         scheduler=scheduler1,
                         epochs=epochs
                         )

# 첫번쨰 모델에 최적 가중치 적용

model_list[0].load_state_dict(model_state_dict)

에폭 [1/20] 
---------------------


  0%|          | 0/250 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: './input/COVID_19_XRAY/train/covid\\image_1962.png'